<a href="https://colab.research.google.com/github/ck1972/Hands-On-GeoAI1/blob/main/GeoAI_Lab_1_Getting_Started_with_Earth_Engine_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1. Getting Started with Earth Engine in Python**
## Introduction
This lab introduces participants to the Google Earth Engine (GEE) Python API for accessing, filtering, visualizing, and analyzing satellite imagery. It provides hands-on experience with remote sensing datasets from multiple satellite missions. It helps participants become comfortable loading geographic boundaries, selecting image collections, and displaying satellite composites interactively using the geemap library.

### Aim
To provide a practical introduction to the GEE Python API and build foundational skills in accessing and visualizing multi-sensor Earth observation data for geospatial analysis.

### Objectives
By the end of this lab, participants will be able to:
*   Authenticate and initialize the GEE Python API using geemap.
*   Load and filter administrative boundary datasets using GeoBoundaries.
*   Access and visualize multispectral and radar satellite data, including Sentinel-2, Sentinel-1, Landsat 9, and ALOS PALSAR-2.
*   Visualize ESA WorldCover land cover maps with a legend.

# Initialize and Authenticate Earth Engine
To get started with Google Earth Engine (GEE), you need to initialize and authenticate the Earth Engine API. Follow these steps.


First, import the Earth Engine API by importing the ee module into your Python environment. This module allows you to interact with the Earth Engine platform.


In [1]:
# Import the API
import ee

# Import the geemap library
import geemap

Next, initialize the Earth Engine API. You must initialize the API to use Earth Engine functionalities. This involves authenticating your session and initializing the library. When you run the ee.Initialize() command for the first time, you might be prompted to authenticate your session. This will open a web browser window where you need to log in with your Google account and grant Earth Engine access.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-kamusoko-test') # Change to your EE project

## Define the boundary
First, we define an area of interest by creating a boundary using the Comprehensive Global Administrative Zones (CGAZ) dataset from the
awesome-gee-community-catalog (https://gee-community-catalog.org/projects/geoboundary/).

In [ ]:
# Load the GeoBoundaries administrative boundaries datasets
admin0 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM0")
admin1 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1")
admin2 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2")

# Check available country names (run this once to inspect)
country_names = admin0.aggregate_array('shapeName').distinct().getInfo()
print('Country names:', country_names)

# Filter the admin0 collection for area of interests (e.g., country) using the exact name.
# Update the name as needed based on the country name in the dataset.
admin0 = admin0.filter(ee.Filter.eq('shapeName', 'Malaysia'))
print('Admin0 for Malaysia count:', admin0.size().getInfo())

# Get the Country's unique shapeID from the admin0 collection
shape_id = ee.Feature(admin0.first()).get('shapeID')
print('Malaysia shapeID:', shape_id.getInfo())

# Filter the admin1 collection for features belonging to country using the ADM0_shape field
admin1 = admin1.filter(ee.Filter.eq('ADM0_shape', shape_id))
print('Admin1 for Malaysia count:', admin1.size().getInfo())

# Filter the admin2 collection for features that belong to Malaysia based on the admin1 shapeIDs
admin1_ids = admin1.aggregate_array('shapeID')
admin2 = admin2.filter(ee.Filter.inList('ADM1_shape', admin1_ids))
print('Admin2 for Malaysia count:', admin2.size().getInfo())

# Create an interactive map centered on Sri Lanka (approximate center)
Map = geemap.Map(center=[1.4854, 103.7618], zoom=6)

# Add layers to the map
Map.addLayer(admin0, {'color': 'red'}, 'Malaysia Admin0')
Map.addLayer(admin1, {'color': 'blue'}, 'Malaysia Admin1')
Map.addLayer(admin2, {'color': 'green'}, 'Malaysia Admin2')

# Display the map
Map

## Display Johor State boundary
Next, load the administrative boundaries from the GeoBoundaries dataset, filters for Johor State in Malaysia, and visualizes its boundary on an interactive map using the geemap library.

In [ ]:
# Load the GeoBoundaries administrative boundaries datasets
admin1 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1")

# Filter the admin2 collection.
# You might need to adjust the name according to the dataset.
boundary = admin1.filter(ee.Filter.eq('shapeName', 'Johor'))

# Check the count of administrative regions for Colombo
print('Admin1 count:', boundary.size().getInfo())

# Create an interactive map centered on Colombo (approximate center)
Map = geemap.Map(center=[1.4854, 103.7618], zoom=9)

# Add the Colombo admin2 layer to the map
Map.addLayer(boundary, {'color': 'green'}, 'Johor')

# Display the map
Map

## Visualize Sentinel-2 RGB Imagery over Johor
Next, filter Sentinel-2 surface reflectance imagery for the year 2024 over Johor State, creates a median composite, and displays a true-color RGB image on the interactive map.

In [ ]:
# Load and Filter Sentinel-2 ===
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(boundary) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .median() \
    .clip(boundary)

s2_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # RGB
}
Map.addLayer(s2, s2_vis, 'Sentinel-2 RGB (2023)')

# Display the map
Map

## Visualize Sentinel-1 VV, VH, and composite backscatter
Next, process Sentinel-1 radar imagery for 2024 over Johor, displaying the VV and VH polarization bands individually and as a pseudo-RGB composite.

In [ ]:
# Load and Filter Sentinel-1
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(boundary) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .median() \
    .clip(boundary)

# Display VV band only
Map.addLayer(s1.select('VV'), {'min': -20, 'max': 0}, 'Sentinel-1 VV')

# Display VH band only
Map.addLayer(s1.select('VH'), {'min': -25, 'max': -5}, 'Sentinel-1 VH')

# Display VV/VH Composite (VV-R, VH-G, VV-B)
composite_vis = {
    'min': -25,
    'max': 0,
    'bands': ['VV', 'VH', 'VV']  # Pseudo-RGB
}
Map.addLayer(s1, composite_vis, 'Sentinel-1 VV/VH Composite')

# Show map
Map

## Create ALOS PALSAR-2 ScanSAR
Next, disply ALOS PALSAR-2 (Advanced Land Observing Satellite – Phased Array type L-band Synthetic Aperture Radar). ALOS PALSAR-2  is a Japanese L-band radar sensor (Japan Aerospace Exploration Agency).

In [ ]:
# Load ALOS PALSAR-2 ScanSAR image collection and filter by boundary and date
collection = (
    ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR')
    .filterBounds(boundary)
    .filterDate('2024-01-01', '2024-12-31')
)

# Compute median composites for HH and HV
Palsar_median_HH = collection.select('HH').median().clip(boundary).unitScale(0, 8000)

# Add PALSAR ScanScar layers to the map
Map.addLayer(Palsar_median_HH, {'min': 0, 'max': 1}, 'PALSAR Median HH')

# Show map
Map

## Display ESA land cover
Finally, display the ESA land cover map for 2021.

In [ ]:
# Load ESA WorldCover Land Cover
esa_lc = ee.Image("ESA/WorldCover/v200/2021").clip(boundary)

# Define Visualization Parameters
lc_vis = {
    'min': 10,
    'max': 100,
    'palette': [
        '006400', 'ffbb22', 'ffff4c', 'f096ff', 'fa0000',
        'b4b4b4', 'f0f0f0', '0064c8', '0096a0', '00cf75',
        'fae6a0', '58481f', '0096c8', 'c8c8c8', '677819',
        'dcd159', 'ff6d4c'
    ]
}

# Add Land Cover Layer to Map
Map.addLayer(esa_lc, lc_vis, 'ESA WorldCover (2021)')

# Add Legend
legend_dict = {
    'Tree cover': '006400',
    'Shrubland': 'ffbb22',
    'Grassland': 'ffff4c',
    'Cropland': 'f096ff',
    'Built-up': 'fa0000',
    'Bare / sparse vegetation': 'b4b4b4',
    'Snow and ice': 'f0f0f0',
    'Permanent water bodies': '0064c8',
    'Herbaceous wetland': '0096a0',
    'Mangroves': '00cf75',
    'Moss and lichen': 'fae6a0',
    'Sparse trees': '58481f',
    'Sparse shrubland': '0096c8',
    'Sparse herbaceous': 'c8c8c8',
    'Bare soil': '677819',
    'Open rock': 'dcd159',
    'Urban green space': 'ff6d4c',
}

Map.add_legend(title="ESA WorldCover 2021", legend_dict=legend_dict)

# Display the Interactive Map
Map

## Lab Assignment
### Task
Using your study area of interest (country, province, or district), perform the following:
1. Display the boundary of your chosen administrative unit using GeoBoundaries.
2. Load and visualize the following Earth observation datasets clipped to your boundary:
      * Landsat 9 Surface Reflectance (2023 or 2024)
      * Sentinel-2 Surface Reflectance (RGB composite)
      * Sentinel-1 SAR (VV and VH bands + composite)
      * ALOS PALSAR-2 ScanSAR (HH band)
      * ESA WorldCover Land Cover Map (2021)
      * Add a legend to the map for the ESA land cover classes.
3. Create a list of relevant land cover classes for your study area (e.g., forest, built-up, cropland, water) based on the WorldCover map.   List item
4. Save your script as a Python notebook (.ipynb) or Python script (.py) and
upload a screenshot of your map output.